# ⚽⚾🥎 **HOL: Olympic Games Data Hub** 🥇🥈🥉

---

Welcome to the **Olympic Games Data Hub**! In this notebook, we will use Python functions and External Access Integration to load and analyze data about the Olympics from 1896 to 2022. Our data source is a GitHub repository, from which we'll fetch and directly store historical Olympic data in our Snowflake account. No S3 buckets or local downloads are needed — our goal is to simplify the execution of this Hands-On Lab (HOL) while showcasing the extensive capabilities of Snowflake!

### What We'll Do:
1. **Load Data**: Fetch Olympic data from the GitHub repository.
2. **Analyze Data**: Utilize Snowpark DataFrames for data preparation and analysis.
3. **Visualize Insights**: Build interactive dashboards with Streamlit for comprehensive analytics.

![Olympic Rings](https://upload.wikimedia.org/wikipedia/commons/thumb/5/5c/Olympic_rings_without_rims.svg/640px-Olympic_rings_without_rims.svg.png)

---

Dive into the code below to start exploring and analyzing the fascinating world of the Olympics!


### Setup

Before using this notebook, ensure that you have created the following objects by running the `setup.sql` script in a worksheet:

- **Database**: `OLYMPIC_GAMES`
- **Schema**: `RAW_DATA`
- **Warehouse**: `OLYMPICS_GAMES_WH`
- **Network Rule**: `GITHUB_NETWORK_RULE`
- **External Access Integration**: `GITHUB_EXTERNAL_ACCESS_INTEGRATION`

The first three items are required as you will need to define the Database, Schema, and Warehouse when you import this notebook into the Snowflake UI.

For the **Network Rule** and **External Access Integration**, once created, follow these steps to make them available within this notebook:

1. **Click on Notebook Settings** (located at the top right of the worksheet screen).
2. **Select the External Access Tab**.
3. **Enable** `GITHUB_EXTERNAL_ACCESS_INTEGRATION` from the list.
4. **Reload the Notebook**. Once reloaded, you will have access to the GitHub URL directly from this notebook.

With these configurations in place, you’ll be ready to extract and work with the dataset from the external GitHub URL in the following cells.

---


In [ ]:
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark import Session

session = get_active_session()

In [ ]:
--Note: For this Hands-On Lab (HOL), we are not creating ad hoc roles and users to minimize prerequisites and simplify setup.

USE ROLE ACCOUNTADMIN;

In [ ]:
import requests
import pandas as pd
from snowflake.snowpark import DataFrame as df
from io import StringIO

def fetch_dataset_from_github(url: str) -> 'DataFrame':
    # Fetch the CSV data from the URL
    response = requests.get(url)
    if response.status_code == 200:
        # Decode the content and read into a Pandas DataFrame
        csv_data = response.content.decode('utf-8')
        csv_file = StringIO(csv_data)
        pandas_df = pd.read_csv(csv_file)
        
        # Convert Pandas DataFrame to Snowpark DataFrame
        return session.create_dataframe(pandas_df)
    else:
        raise Exception(f"Failed to fetch CSV: {response.status_code} - {response.text}")


In [ ]:
from snowflake.snowpark.functions import col

# Function to rename columns to uppercase
def rename_columns_to_uppercase(df):
    # Generate a list of columns with uppercase names
    new_columns = [col(c).alias(c.upper()) for c in df.columns]
    # Select columns with new names
    return df.select(*new_columns)

# Base URL and list of files
url_base = 'https://github.com/sfc-gh-mconsoli/olympic_games_data_hub/raw/main/dataset/'
url_files = [
    'Olympic_Games.csv',
    'Olympic_Athlete_Bio.csv',
    'Olympic_Results.csv',
    'Olympic_Athlete_Event_Results.csv',
    'Olympic_Country.csv',
    'Olympic_Games_Medal_Tally.csv'
]

# Loop through each URL
for url in url_files:
    # Get Snowpark DataFrame from the URL
    df = fetch_dataset_from_github(url_base + url)

    # Extract table name from URL
    table_name = url.split('/')[-1].replace('.csv', '').upper()

    # Drop the table if it exists
    session.sql(f"DROP TABLE IF EXISTS {table_name}").collect()

    # Convert column names to uppercase
    df = rename_columns_to_uppercase(df)

    # Create table and insert data from Snowpark DataFrame
    df.write.save_as_table(table_name, mode='overwrite')

    print(f"Table {table_name} created and data loaded successfully.")


In [ ]:
# Let's check loaded data

session.table('OLYMPIC_GAMES_MEDAL_TALLY').limit(51)

In [ ]:
-- No surprise, the tallest are the basketball players! :D 
SELECT * FROM 
OLYMPIC_GAMES.RAW_DATA.OLYMPIC_ATHLETE_BIO
WHERE COUNTRY like '%Italy%' and HEIGHT > 180 
order by HEIGHT DESC
LIMIT 50

In [ ]:
import streamlit as st
import snowflake.snowpark as sp

# Load the Snowpark DataFrame from the table
df = session.table('OLYMPIC_GAMES_MEDAL_TALLY')

# Aggregate gold medals per country
gold_medals_df = df.group_by("COUNTRY").agg(sp.functions.sum("GOLD")
                .alias("TOTAL_GOLD_MEDALS")).limit(10)

# Collect the data as a list of dictionaries
data = gold_medals_df.collect()
data_list = [row.as_dict() for row in data]

# Convert the list of dictionaries into two lists: one for labels and one for values
countries = [row['COUNTRY'] for row in data_list]
gold_medals = [row['TOTAL_GOLD_MEDALS'] for row in data_list]


In [ ]:
import plotly.graph_objects as go

# Streamlit app
st.title("Olympic Games Medal Tally")

# Pie chart using Plotly
st.subheader("Countries with the Most Gold Medals")
fig = go.Figure(data=[go.Pie(labels=countries, values=gold_medals, hole=0.3)])
fig.update_layout(margin=dict(t=0, b=0, l=0, r=0), title_text="Gold Medals by Country")
st.plotly_chart(fig)

### 🚀 Elevate Your Experience

This notebook has been great so far, but let’s take it to the next level! In the following steps, we will integrate the Streamlit app into Snowflake. Here’s how you can do it:

1. **Download the Streamlit App Script**
   - Get the [`olympic_games_data_hub.py`](https://github.com/sfc-gh-mconsoli/olympic_games_data_hub) file from the GitHub repository.

2. **Create a Streamlit App in Snowsight Projects**
   - Navigate to **Snowsight Projects**.
   - Create a new **Streamlit App** and import the downloaded file.

3. **Explore and Analyze**
   - Once imported, the app will be ready to go!
   - Feel free to dive deeper into the data, either through this notebook or by exploring the Streamlit app.

Enjoy exploring and analyzing the Olympic Games data!
